In [9]:
import numpy as np 
import healpy as hp
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm

In [10]:
nside = 256
lmax = 3*nside-1
ell_input, TT, TE, EE, BB, PP = np.loadtxt('cl.txt', unpack=True)
input_cls = np.array([TT, EE, BB, TE]) 
input_cls /= ell_input*(ell_input+1)/2/np.pi 
for c in input_cls: c[0] = 0 
input_cls = input_cls[:,:lmax+1]

input_cls_noBB = np.copy(input_cls)
input_cls_noBB[2] = np.zeros(lmax+1)

input_cls_onlyBB = np.copy(input_cls)
input_cls_onlyBB[0] = np.zeros(lmax+1)
input_cls_onlyBB[1] = np.zeros(lmax+1)
input_cls_onlyBB[3] = np.zeros(lmax+1)

/tmp/ipykernel_5771/3975758150.py:5: RuntimeWarning: invalid value encountered in divide
  input_cls /= ell_input*(ell_input+1)/2/np.pi


In [11]:
nmc = 100
fwhm = np.deg2rad(17/60) #sat 145ghz
for i in tqdm(range(nmc)):
    np.random.seed(i)
    input_map = hp.synfast(input_cls, nside, fwhm=fwhm,new=True)
    hp.write_map(f'lcdm_maps/map_{i}.fits', input_map, overwrite=True)

    input_map = hp.synfast(input_cls_onlyBB, nside, fwhm=fwhm, new=True)
    hp.write_map(f'lcdm_BBonly_maps/map_{i}.fits', input_map, overwrite=True)

  0%|                                                   | 0/100 [00:00<?, ?it/s]setting the output map dtype to [dtype('float64'), dtype('float64'), dtype('float64')]
setting the output map dtype to [dtype('float64'), dtype('float64'), dtype('float64')]
  1%|▍                                          | 1/100 [00:00<01:16,  1.29it/s]setting the output map dtype to [dtype('float64'), dtype('float64'), dtype('float64')]
setting the output map dtype to [dtype('float64'), dtype('float64'), dtype('float64')]
  2%|▊                                          | 2/100 [00:01<01:08,  1.42it/s]setting the output map dtype to [dtype('float64'), dtype('float64'), dtype('float64')]
setting the output map dtype to [dtype('float64'), dtype('float64'), dtype('float64')]
  3%|█▎                                         | 3/100 [00:02<01:06,  1.47it/s]setting the output map dtype to [dtype('float64'), dtype('float64'), dtype('float64')]
setting the output map dtype to [dtype('float64'), dtype('float64'), dt